In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm

from collections import Counter

unable to import 'smart_open.gcs', disabling that module


In [2]:
def delete_punctuation_marks_tag(word):
    word = word.replace(",","")
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("!","")
    word = word.replace(":","")
    word = word.replace("?","")
    word = word.replace("~","")
    word = word.replace("\\","")
    word = word.replace("\"","")
    word = word.replace(";","")
    return word.lower()

In [3]:
eat_info = pd.read_csv('../data/translated_eat_info.csv')
eat_review = pd.read_csv('../data/translated_eat_review.csv')

In [4]:
eat_review_by_user = pd.DataFrame(index = eat_review['u_id'].unique(), columns=['reviews'])

for id in tqdm(eat_review['u_id'].unique()):
    review_all = ''
    for review in eat_review.loc[eat_review['u_id']==id,'review_eng']:
        if type(review) != float:
            review_all += review+'. '
    eat_review_by_user.loc[id,'reviews'] = review_all

In [5]:
eat_info['tags']=''
eat_review_by_user['tags']=''

In [6]:
eat_info.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags,p_id,reviews
0,놀부만두,서울특별시 동대문구 휘경2동 276-33,NaN,냉모밀,6000.0,"{'월-금,일': '오전 11시 - 오후 8시 30분 '}",NaN,NaN,,721,"As a local restaurant, it was moderately shabb..."
1,민벅 건대점,서울특별시 광진구 화양동 10-1,NaN,알리오올리오,9900.0,{'매일': '오전 11시 30분 - 오후 10시 (구정 추석 당일 3시 오픈)'},NaN,NaN,,1832,"If you order 1 menu per person, give me Gorgon..."
2,뉴델리,서울시 동대문구 회기동 3-182,NaN,치킨 탄두리 (한마리),15000.0,{'매일': '오전 11시 - 오후 10시 '},NaN,NaN,,741,You can enjoy delicious Indian curry at an old...
3,강릉스낵,서울특별시 양천구 목동 406-65,NaN,직송 모듬 회,60000.0,"{'월-금': '오후 5시 - 오전 1시 ', '토/일': '오후 4시 - 오전 1...",NaN,NaN,,218,Tteokbokki is spicy and delicious. Fried chick...
4,여기가전집,서울특별시 구로구 구로3동 1124-34,NaN,모둠전,18000.0,{'월-금': '오후 2시 - 오전 2시 '},NaN,NaN,,3276,"The pancakes are delicious~ Potato pancake, pe..."


In [7]:
eat_review_by_user.head()

,reviews,tags
5351,"As a local restaurant, it was moderately shabb...",
3024,I went to Pyongyang naengmyeon in Jegi-dong an...,
5941,"Among the three, kimchi dumplings were the bes...",
13580,The combination of kimchi dumplings and cold b...,
6953,,


In [8]:
with open('../data/stop_words.txt','r',-1,'utf-8') as f:
    stop_words = set(f.read().split('/'))

In [9]:
stop_words

{'',
 '*',
 '+',
 '+0',
 '+_+',
 '0th',
 '1',
 '1st',
 '2',
 '2nd',
 '3',
 '3rd',
 '4',
 '4b',
 '4th',
 '5th',
 '6',
 '6th',
 '7th',
 '8th',
 '9',
 '=',
 '>',
 '@',
 '__',
 '___________________________________',
 'a',
 'a4',
 'about',
 'above',
 'accumulated',
 'acrostic',
 'ade',
 'admirable',
 'adzuki',
 'after',
 'again',
 'against',
 'ago',
 'agre',
 'ah',
 'ahead',
 'ai',
 'ain',
 'ajumma',
 'aka',
 'al',
 'albert',
 'all',
 'alla',
 'alle',
 'alli',
 'allthe',
 'alma',
 'alongthe',
 'altro',
 'am',
 'amazing',
 'ame',
 'americano',
 'amo',
 'an',
 'anam',
 'ancestral',
 'ancient',
 'and',
 'anniversary',
 'annual',
 'ant',
 'anticipated',
 'any',
 'apo',
 'arabian',
 'archa',
 'are',
 'aren',
 "aren't",
 'argent',
 'arti',
 'as',
 'asu',
 'at',
 'attempted',
 'austrian',
 'avai',
 'average',
 'avo',
 'awesome',
 'b',
 'baby',
 'badam',
 'badi',
 'bahn',
 'bal',
 'baltic',
 'bam',
 'bb',
 'be',
 'bean',
 'beans',
 'because',
 'been',
 'beep',
 'bef',
 'before',
 'beforethe',
 'beg

In [10]:
def get_tags_to_txt(text):
    if type(text) != str:
        return ''
    text = word_tokenize(text)
    tags_dump = nltk.pos_tag(text)
    tags = []
    
    for word in tags_dump:
        if (word[1] == 'JJ') and (word[0] not in stop_words):
            tags.append(delete_punctuation_marks_tag(word[0]))
            
    tags = ','.join(tags)
    
    return tags

In [11]:
for i in tqdm(eat_info.index):
    eat_info.loc[i,'tags'] = get_tags_to_txt(eat_info.loc[i,'reviews'])
    
for i in tqdm(eat_review_by_user.index):
    eat_review_by_user.loc[i,'tags'] = get_tags_to_txt(eat_review_by_user.loc[i,'reviews'])

In [12]:
w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [13]:
All_tags = set()
for tags in eat_info['tags']:
    All_tags.update(set(tags.split(',')))

In [14]:
new_df = pd.DataFrame()
for word in All_tags:
    try:
        f_vec = w2v.get_vector(word)
        new_df[word] = f_vec
    except:
        pass

In [15]:
x = new_df.T
cluster_count=120
model = KMeans(n_clusters=cluster_count)
model.fit(x)
model.predict(x)
Y=x.copy()
Y['kmeans_id'] = model.predict(x)

In [16]:
km = list(Y['kmeans_id'])
max_count = 0
max_count_id = ''
for i in range (0, cluster_count):
    if km.count(i) > max_count:
        max_count = km.count(i);
        max_count_id = i;
    print(i,': ', km.count(i))
max_count_id

0 :  2
1 :  113
2 :  36
3 :  254
4 :  43
5 :  1
6 :  43
7 :  4
8 :  60
9 :  47
10 :  21
11 :  72
12 :  11
13 :  117
14 :  42
15 :  111
16 :  19
17 :  23
18 :  34
19 :  59
20 :  73
21 :  7
22 :  129
23 :  59
24 :  36
25 :  85
26 :  17
27 :  34
28 :  13
29 :  25
30 :  47
31 :  37
32 :  40
33 :  54
34 :  82
35 :  3
36 :  59
37 :  55
38 :  3
39 :  65
40 :  40
41 :  24
42 :  153
43 :  41
44 :  14
45 :  71
46 :  17
47 :  27
48 :  29
49 :  3
50 :  7
51 :  54
52 :  24
53 :  29
54 :  150
55 :  47
56 :  45
57 :  19
58 :  55
59 :  31
60 :  63
61 :  43
62 :  33
63 :  3
64 :  53
65 :  53
66 :  118
67 :  31
68 :  28
69 :  61
70 :  227
71 :  6
72 :  32
73 :  43
74 :  73
75 :  25
76 :  10
77 :  28
78 :  47
79 :  53
80 :  57
81 :  16
82 :  33
83 :  45
84 :  31
85 :  31
86 :  2
87 :  51
88 :  3
89 :  10
90 :  89
91 :  57
92 :  26
93 :  161
94 :  29
95 :  29
96 :  21
97 :  34
98 :  4
99 :  41
100 :  39
101 :  12
102 :  22
103 :  91
104 :  32
105 :  25
106 :  20
107 :  60
108 :  31
109 :  105
110 :  56
11

3

In [17]:
km_df = pd.DataFrame()
km_df['group_'+str(max_count_id)] = pd.Series(list(Y[Y['kmeans_id'] == max_count_id].index))
for i in range(0, cluster_count):
    col_name = 'group_'+str(i)
    words = list(Y[Y['kmeans_id'] == i].index)
    km_df[col_name] = pd.Series(words)

In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
km_df.fillna(0)

,group_3,group_0,group_1,group_2,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,backward,scramble,complemented,hungry,ha,unsweetened,vogue,ruby,vertical,fin,satisfied,hourly,peas,residual,soggy,make,miso,amino,spicy,overwhelming,hearted,horse,pad,exam,potted,swag,proportioned,fattening,culinary,overcooked,weird,duet,smart,heretic,carbonic,rechargeable,ciabatta,stingy,entangled,fame,alertness,thawed,embryonic,pulse,crust,manageable,candlelight,negligent,neighborly,whale,hygienic,baseless,pub,expressionless,striking,tepid,hair,mug,respectful,landscaping,dangerous,bone,mexican,funeral,squirrel,sprite,designated,pungent,redecorated,playful,which,coma,fluffy,clean,miserable,crunchy,harmony,homey,happiest,strawberry,ubiquitous,drinkable,basil,stationery,gastric,filthy,capitalistic,masala,moisturizing,admissible,wrinkled,scones,shellfish,base,buckwheat,blood,sonic,tighter,delayed,milkshake,unexpressed,toy,veal,napoleon,fierce,corona,urethane,immaculate,cupboard,stable,incomplete,unofficial,anxious,pheasant,stimuli,chewed,curled,finn,ethnic,worm
1,up,scrambled,paired,tired,iti,0,stylish,pearl,millimeter,lake,impressed,corresponding,vegetables,inducing,hazy,present,garnish,enzyme,sweet,impressive,snobbish,oxen,door,pupils,pine,hate,naturalistic,meatless,chefs,thickened,pretentious,indie,unconventional,traditionalist,gel,battery,dish,undervalued,tangled,visa,loveliness,thawing,unexplored,consciousness,meringue,stimulating,meal,unauthorized,cooperative,stingray,unsanitary,defenseless,bakery,cried,neutralizing,unapproachable,pant,thermos,obedient,marbled,taboo,legs,african,memorial,humanely,flowery,toldthe,aromatic,ported,sentimental,wellas,unresponsive,saccharine,trash,crumbling,parfait,inseparable,paradise,adoring,durian,numbered,liqueur,dill,walnut,ovarian,clogged,capitalist,kebab,moisturized,copied,hairy,yogurt,salmon,hadi,granary,splashing,lyrical,cleaner,canceled,cocktails,nostalgia,marshmallows,tenderloin,roast,bitter,equatorial,matte,dazzling,oven,popular,irregular,travel,shy,gull,0,chew,broke,vietnam,military,bug
2,depite,0,layered,bored,ini,0,modern,jade,airflow,ratfish,pleased,low,tomatoes,heavy,cooler,find,mayo,xylitol,savored,unparalleled,lazy,foreleg,starter,learning,banyan,haha,tidy,cardio,barista,crumbly,whiny,horn,aggressive,yogi,phosphoric,refillable,appetizer,adequate,tangle,degree,stubbornness,chill,unpleasing,feeling,runny,accessible,brunch,abusive,welcoming,shark,cleanliness,insane,cafe,fist,floating,useless,chiffon,cups,considerate,mural,stressful,niggle,japanese,burial,yak,sensual,asked,jasmine,remodeled,witty,global,intoxicated,grue,styrofoam,disappointing,mocha,unite,pleasant,likes,mint,molded,rosé,olives,charcoal,intestinal,dingy,0,yakiniku,hydrated,sourced,yellowish,fondue,scallops,importanti,milk,dried,accompaniment,fresher,delay,bottled,dislike,balloon,wagyu,pickle,tense,lunar,varnish,gem,knife,indispensable,vague,festival,reluctant,quail,0,chews,nibbled,ame,economic,spam
3,mean,0,reflective,disheveled,galat,0,class

In [19]:
# remove groups we don't need
# rm_groups = [17, 26, 50, 63]
# for i in rm_groups:
#     group_name = 'group_'+str(i);
#     stop_words.update(set(km_df[group_name].dropna()))
# with open('../data/stop_words.txt','w',-1,'utf-8') as f:
#     f.write('/'.join(stop_words))

In [20]:
eat_info.set_index(eat_info['p_id'],inplace=True,drop=True)
del eat_info['p_id']
eat_info.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags,reviews
p_id,,,,,,,,,,
721,놀부만두,서울특별시 동대문구 휘경2동 276-33,NaN,냉모밀,6000.0,"{'월-금,일': '오전 11시 - 오후 8시 30분 '}",NaN,NaN,"local,shabby,small,deep,thin,bite-sized,greasy...","As a local restaurant, it was moderately shabb..."
1832,민벅 건대점,서울특별시 광진구 화양동 10-1,NaN,알리오올리오,9900.0,{'매일': '오전 11시 30분 - 오후 10시 (구정 추석 당일 3시 오픈)'},NaN,NaN,"strong,delicious,fresh,delicious,high,small,ea...","If you order 1 menu per person, give me Gorgon..."
741,뉴델리,서울시 동대문구 회기동 3-182,NaN,치킨 탄두리 (한마리),15000.0,{'매일': '오전 11시 - 오후 10시 '},NaN,NaN,"delicious,indian,old,indian,long,exotic,genera...",You can enjoy delicious Indian curry at an old...
218,강릉스낵,서울특별시 양천구 목동 406-65,NaN,직송 모듬 회,60000.0,"{'월-금': '오후 5시 - 오전 1시 ', '토/일': '오후 4시 - 오전 1...",NaN,NaN,"delicious,crispy,light,quick,delicious,light,s...",Tteokbokki is spicy and delicious. Fried chick...
3276,여기가전집,서울특별시 구로구 구로3동 1124-34,NaN,모둠전,18000.0,{'월-금': '오후 2시 - 오전 2시 '},NaN,NaN,"delicious,bad,,half-and-half,delicious,fine,","The pancakes are delicious~ Potato pancake, pe..."


In [21]:
res_matrix = pd.DataFrame(index=eat_info.index,columns=list(km_df.columns))
user_matrix = pd.DataFrame(index=eat_review_by_user.index,columns=list(km_df.columns))
res_matrix = res_matrix.fillna(0)
user_matrix = user_matrix.fillna(0)

In [22]:
res_matrix.head()

,group_3,group_0,group_1,group_2,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
p_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
721,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1832,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
741,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3276,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
user_matrix.head()

,group_3,group_0,group_1,group_2,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
5351,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13580,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
for id in tqdm(res_matrix.index):
    tags = Counter(eat_info.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                res_matrix.loc[id,'group_'+str(num)] += tags[tag]

In [25]:
res_matrix.head()

,group_3,group_0,group_1,group_2,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
p_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
721,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1832,3,0,0,0,0,0,0,0,0,0,1,2,0,1,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,4,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
741,8,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
218,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3276,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
for id in tqdm(user_matrix.index):
    tags = Counter(eat_review_by_user.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                user_matrix.loc[id,'group_'+str(num)] += tags[tag]

In [27]:
user_matrix.head()

,group_3,group_0,group_1,group_2,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
5351,7,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,11,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2,0,1,0,0,5,0,0,4,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0
3024,36,0,8,0,0,0,0,0,0,2,8,4,0,5,11,1,6,0,48,4,0,0,5,0,0,0,0,1,0,6,0,0,1,0,2,0,0,1,0,4,0,8,1,0,0,11,0,0,1,0,0,1,1,0,2,0,0,0,1,2,5,0,10,0,0,0,0,0,0,0,13,0,9,8,4,0,0,2,0,0,1,0,6,0,0,0,0,6,0,0,19,0,2,9,1,1,0,0,0,0,0,0,0,5,3,0,0,0,0,15,0,2,1,0,0,0,0,0,0,0
5941,36,0,6,1,0,0,1,0,0,0,3,2,0,12,3,0,0,0,106,6,0,0,5,1,0,1,0,1,0,2,4,0,3,1,1,0,1,2,0,2,0,2,1,1,0,22,0,0,0,0,0,0,1,0,1,3,0,0,3,0,6,0,3,0,0,2,1,0,1,0,11,0,10,6,17,1,0,4,0,5,0,0,6,0,0,4,0,7,0,0,24,1,1,38,0,0,0,0,0,0,0,0,0,5,2,0,0,0,0,33,0,3,0,0,0,0,1,1,0,0
13580,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,3,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
user_matrix.head()

,group_3,group_0,group_1,group_2,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_64,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
5351,7,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,11,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2,0,1,0,0,5,0,0,4,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0
3024,36,0,8,0,0,0,0,0,0,2,8,4,0,5,11,1,6,0,48,4,0,0,5,0,0,0,0,1,0,6,0,0,1,0,2,0,0,1,0,4,0,8,1,0,0,11,0,0,1,0,0,1,1,0,2,0,0,0,1,2,5,0,10,0,0,0,0,0,0,0,13,0,9,8,4,0,0,2,0,0,1,0,6,0,0,0,0,6,0,0,19,0,2,9,1,1,0,0,0,0,0,0,0,5,3,0,0,0,0,15,0,2,1,0,0,0,0,0,0,0
5941,36,0,6,1,0,0,1,0,0,0,3,2,0,12,3,0,0,0,106,6,0,0,5,1,0,1,0,1,0,2,4,0,3,1,1,0,1,2,0,2,0,2,1,1,0,22,0,0,0,0,0,0,1,0,1,3,0,0,3,0,6,0,3,0,0,2,1,0,1,0,11,0,10,6,17,1,0,4,0,5,0,0,6,0,0,4,0,7,0,0,24,1,1,38,0,0,0,0,0,0,0,0,0,5,2,0,0,0,0,33,0,3,0,0,0,0,1,1,0,0
13580,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,3,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
km_df.count()

group_3      254
group_0        2
group_1      113
group_2       36
group_4       43
group_5        1
group_6       43
group_7        4
group_8       60
group_9       47
group_10      21
group_11      72
group_12      11
group_13     117
group_14      42
group_15     111
group_16      19
group_17      23
group_18      34
group_19      59
group_20      73
group_21       7
group_22     129
group_23      59
group_24      36
group_25      85
group_26      17
group_27      34
group_28      13
group_29      25
group_30      47
group_31      37
group_32      40
group_33      54
group_34      82
group_35       3
group_36      59
group_37      55
group_38       3
group_39      65
group_40      40
group_41      24
group_42     153
group_43      41
group_44      14
group_45      71
group_46      17
group_47      27
group_48      29
group_49       3
group_50       7
group_51      54
group_52      24
group_53      29
group_54     150
group_55      47
group_56      45
group_57      19
group_58      

In [30]:
eat_review_score = pd.DataFrame(eat_review, columns=['u_id','p_id','rating'])
eat_review_score.sort_values(by=['u_id'], axis=0,inplace=True)
eat_review_score.reset_index(drop = True, inplace=True)
eat_review_score.head()

,u_id,p_id,rating
0,0,3375,3
1,0,2778,5
2,0,612,5
3,0,3949,5
4,0,1679,1


In [32]:
res_matrix.to_csv('../data/res_matrix.csv')
user_matrix.to_csv('../data/user_matrix.csv')
eat_review_score.to_csv('../data/score_board.csv',index=False)
km_df.to_csv('../data/eat_cluster.csv',index=False)
km_df.count().to_csv('../data/group_count.csv')

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """
